In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Logistic Model

In [ ]:
# # y_hat = sigmoid(w.X + b)
# # from tqdm import tqdm


# logistic = lambda z: 1./ (1 + np.exp(-z))
# class LogisticRegression:  
#   """
#   M: Number of classes
#   D: Feature Dimension
#   N: Number of rows
#   """
#   def __init__(self, learning_rate=.001, max_iters=1e4, eps=1e-8):
#     self.learning_rate = learning_rate
#     self.max_iters = max_iters
#     self.epsilon = eps
    

#   def gradient(self, x, y, params):
#     # assumes x has a bias term (ones)
#     N = x.shape[0] # N number of rows
#     w = params

#     yh = logistic(np.dot(x, w))
#     dy = yh - y
#     dw = np.dot(x.T, dy)/N 
#     dparams = dw
#     return dparams

#   def error(self, x, y, params):
#     y_probs = logistic(np.matmul(x, params))
#     yh = np.argmax(y_probs, axis=1)
#     return (yh != y).sum()/x.shape[0]

#   def fit(self, x, y, k=10):
#     N = x.shape[0]
#     x = np.c_[ x, np.ones(N) ]
#     D = x.shape[1]

#     self.M = len(np.unique(y)) # Number of classes

#     train_idx, valid_idx = [], []

#     # Create Validation and Train sets
#     indices = np.arange(x.shape[0])
#     splits = np.array_split(indices, k)

#     valid_idx = splits[0].tolist()
#     temp_train = []
#     for i in range(1, len(splits)-1):
#       temp_train += splits[i].tolist()

#     x_train, y_train = x[temp_train, :], y[temp_train]
#     x_valid, y_valid = x[valid_idx, :], y[valid_idx]

#     # Encode Y
#     y_train_enc = pd.get_dummies(y_train)

#     # Init parameters
#     params = np.random.randn(D, self.M) * .01
#     self.params = params

#     # Gradient Descent
#     grads_normed = np.array([np.inf]*3)
#     valid_err = [np.inf]
#     # print(self.error(x_valid, y_valid, params))
#     t = 1
# #     pbar = tqdm(total=self.max_iters)
#     while (np.any(grads_normed > self.epsilon) and t < self.max_iters
#         if valid_err[-1] < (self.error(x_valid, y_valid, params)): break
#         self.params = params
#         t += 1
#         valid_err.append(self.error(x_valid, y_valid, params))

#         grad = self.gradient(x_train, y_train_enc, params)
#         params -= self.learning_rate * grad

#         grads_normed = np.array([np.linalg.norm(g) for g in grad])
# #         pbar.update(1)
# #     pbar.close()
# #     print(valid_err)
#     # self.params = self.grad_descent(self.gradient, x_train, y_train_enc, w)


#   def predict(self, x):
#       w = self.params
#       x = np.c_[ x, np.ones(x.shape[0]) ]
#       y_probs = logistic(np.dot(x, w)) 
#       yh = np.argmax(y_probs, axis=1)
#       yh = np.c_[np.arange(0, yh.shape[0], 1, dtype=int), yh]
#       return yh

# Pull Data

In [ ]:
x_train = np.genfromtxt("/kaggle/input/classification-of-mnist-digits/train.csv", delimiter=',', skip_header=1)
x_train = x_train[:, :-1]
# print(x_train.shape)

labels = np.genfromtxt('/kaggle/input/classification-of-mnist-digits/train_result.csv', delimiter=',', skip_header=1)
labels = labels[:, 1]
# print(labels.shape)

x_test = np.genfromtxt('/kaggle/input/classification-of-mnist-digits/test.csv', delimiter=',', skip_header=1)
x_test = x_test[:, :-1]

# Run Logistic


In [ ]:
# log_mod = LogisticRegression()
# log_mod.fit(x_train, labels)
# yh_test = log_mod.predict(x_test)

# LeNet

In [ ]:
def split_train_valid(x, y, k=5):
  # Create Validation and Train sets
    indices = np.arange(x.shape[0])
    splits = np.array_split(indices, k)

    valid_idx = splits[0].tolist()
    temp_train = []
    for i in range(1, len(splits)-1):
      temp_train += splits[i].tolist()
    
    x_train = x[temp_train, :] 
    y_train = y.iloc[temp_train, :]
    x_valid, y_valid = x[valid_idx, :], y.iloc[valid_idx, :]

    return x_train, y_train, x_valid, y_valid

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.optimizers import SGD
# metrics 
from keras.metrics import categorical_crossentropy

def LeNet():
  model = Sequential()
  
  # Convolutional layer  
  model.add(Conv2D(filters = 6, kernel_size = (5,5), padding = 'same', 
                   activation = 'relu', input_shape = (28,28,1)))
  
  # Max-pooing layer with pooling window size is 2x2
  model.add(MaxPooling2D(pool_size = (2,2)))
  
  # Convolutional layer 
  model.add(Conv2D(filters = 16, kernel_size = (5,5), activation = 'relu'))
  
  # Max-pooling layer 
  model.add(MaxPooling2D(pool_size = (2,2)))
  
  # Flatten layer 
  model.add(Flatten())
  
  # The first fully connected layer 
  model.add(Dense(120, activation = 'relu'))
  
  # The output layer  
  model.add(Dense(10, activation = 'softmax'))
  
  # compile the model with a loss function, a metric and an optimizer function
  # In this case, the loss function is categorical crossentropy, 
  # we use Stochastic Gradient Descent (SGD) method with learning rate lr = 0.01 to optimize the loss function
  # metric: accuracy 
  
  opt = SGD(lr = 0.01)
  model.compile(loss = categorical_crossentropy, 
                optimizer = opt, 
                metrics = ['accuracy']) 
                
  return model

In [ ]:
def train_lenet(model, x, y, batch_size=128, epochs = 50):
    '''
    x (_, 28, 28, 1)
    y one hot encoded : (_, 10)

    '''

    # Split Train/Validation
    x_train, y_train, x_valid, y_valid = split_train_valid(x, y)
    # batch_size = 128
    # print(y_valid0.shape)
    # print(y_train0.shape)

    acc = model.fit(x_train, y_train, epochs = epochs,       # HYPERPARAMETER 
                            batch_size = batch_size,             # HYPERPARAMETER
                          steps_per_epoch = x_train.shape[0]//batch_size, 
                          validation_data = (x_valid, y_valid), 
                          validation_steps = x_valid.shape[0]//batch_size, verbose = 1)
    return None

def predict(model, x):
  '''
  input:
  - x: (_, 28, 28, 1)
  output:
  - y: (_, 1)
  '''
  vec_p = model.predict(x)
  # determine the label corresponding to vector vec_p
  y_p = np.argmax(vec_p, axis=1)
#   print(vec_p)
#   print(y_p)
  return y_p

# Run LeNet

In [ ]:
x_train_tens, x_train_ones = x_train[:, :784].reshape(x_train.shape[0], 28, 28, 1), x_train[:, 784:].reshape(x_train.shape[0], 28, 28, 1)
y_train_tens, y_train_ones = labels//10, labels % 10
x_test_tens, x_test_ones = x_test[:, :784].reshape(x_test.shape[0], 28, 28, 1), x_test[:, 784:].reshape(x_test.shape[0], 28, 28, 1)
# print(y_train_tens.shape)

zeros = pd.DataFrame(np.zeros((y_train_tens.shape[0], 8)))
y_train_tens, y_train_ones = pd.concat([pd.get_dummies(y_train_tens), zeros],axis=1), pd.get_dummies(y_train_ones)

In [ ]:
mod_tens = LeNet()
train_lenet(mod_tens, x_train_tens, y_train_tens)
y_tens = predict(mod_tens, x_test_tens)

In [ ]:
mod_ones = LeNet()
train_lenet(mod_ones, x_train_ones, y_train_ones)
y_ones = predict(mod_ones, x_test_ones)

In [ ]:
# Combine y_ones y_tens
y_pred = np.c_[np.arange(y_tens.shape[0]), y_tens*10+y_ones]
np.savetxt("/kaggle/working/submission.csv", y_pred, delimiter=",", header="Index,Class", fmt="%i", comments='')